In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
measles_data = pd.read_csv(
    "./csvs/model_3_1_measles.csv",
    index_col=0,
)
flu_data = pd.read_csv(
    "./csvs/model_3_1_flu.csv",
    index_col=0,
)
config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 10000.,
}
compartments = (
    "Susceptible", 
    "Pre-infectious", 
    "Infectious", 
    "Immune",
)
model = CompartmentalModel(
    times=(0., config["end_time"]),
    compartments=compartments,
    infectious_compartments=["Infectious"],
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infectious_seed"],
        "Infectious": config["infectious_seed"],
    }
)
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("r0") / ave_infous,
    source="Susceptible",
    dest="Pre-infectious"
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
}
model.run(parameters=parameters, solver="euler")
measles_values = model.get_outputs_df()
assert abs(measles_data - measles_values).max().max() < 1e-5
parameters = {
    "r0": 2.,
    "ave_infous": 2.,
    "ave_preinfous": 2.,
}
model.run(parameters=parameters, solver="euler")
flu_values = model.get_outputs_df()
assert abs(flu_data - flu_values).max().max() < 1e-5